# Webhook Example

Webhook pipelines can receive data from a custom webhook while using any sink. This is useful for hooking up a webhook to feed into Kafka or another advanced broker or database.  
It is also good for long-running tasks where we don't want the client to have to wait for the task that they triggered to finish.
### Webhook Pipeline Overview

- **Source:** Listens on a specified port and path for incoming HTTP POST requests (the webhook endpoint).
- **Sink:** Writes received data into a CSV file.

### Imports and Pipeline Setup

In [ ]:
from bspump.jupyter import *
import bspump.http.web.source
import bspump.file
import time
import json

### In Jupyter, you can test the pipeline logic without submitting the form using a sample events:

In [ ]:
sample_events([
    b"""{"foo":"bap"}""",
    b"""{"foo":"baz"}"""
])

### Example pipeline
source and sink registrations

In [ ]:
auto_pipeline(
    source=lambda app, pipeline: bspump.http.web.source.WebHookSource(
       app,
       pipeline,
       config = {
           "port": 8080,
           "path": "/webhook/",
           "secret_qparam": "not-secure",
       }),
    sink=lambda app, pipeline: bspump.file.FileCSVSink(app, pipeline),
    name="WebhookPipeline"
)

```WebHookSource```: Sets up a simple HTTP server at /webhook on port 8080. Requests must include the secret query parameter.

```FileCSVSink```: Every event received is written (appended) as a row to /data/random-numbers-1.csv.

In [5]:
event = json.loads(event)
event

{'foo': 'bap'}
{'foo': 'baz'}


In [6]:
event["foo"] = event["foo"].upper()
event

{'foo': 'BAP'}
{'foo': 'BAZ'}


---
### Testing

You can run the notebook yourself by typing:

```
bitswan notebook WebHooks/main.ipynb
```
In another terminal, you can send an example webhook request like this:
```
curl -X POST \
  'http://localhost:8080/webhook/?secret=not-very-secure' \
  -H 'Content-Type: application/json' \
  -d '{"foo":12345}'

```
Make sure to use the same secret query parameter that your pipeline is expecting.

You should also check the **Secrets/main.ipynb** example to see how you can use secrets to store and manage such sensitive data.